# Imports

In [2]:
import os
from dotenv import load_dotenv

from openai import AsyncOpenAI
from pydantic import BaseModel

from utils.model_costs import ModelUsageAsync
from utils.openai_calls import call_openai_structured

In [3]:
load_dotenv() # .env should be in the root folder (sibling of this notebook)

openai_client = AsyncOpenAI(
    api_key=os.getenv("OPENAI_PROJECT_KEY"),
)

# Using the utils

In [4]:
source_text = """
In the bustling heart of New York City, hidden from ordinary eyes, lay the enchanting neighborhood of Eldergrove, concealed behind an ancient oak in Central Park. Only those with magical sight could see the shimmering gateway, guarded by the wise sorcerer, Professor Isadora Finch.

Maya Torres, a young aspiring magician, recently moved to NYC with her family, unaware of her magical heritage. On her twelfth birthday, Maya received a mysterious invitation, instructing her to touch the oldest oak in Central Park at midnight. Skeptical yet curious, Maya did so and watched in awe as the tree transformed into a glowing portal.

Inside Eldergrove, Maya discovered cobblestone streets bustling with magical beings: talking cats who owned bookstores, pixies selling enchanted pastries, and broomsticks zipping through the air delivering mail. Professor Finch greeted Maya warmly, revealing that her grandmother had once been a powerful magician and it was now Maya's turn to embrace her destiny.

However, danger lurked nearby. A rogue wizard named Elias Grimwald sought to steal the Heartstone—a crystal that powered all magic in Eldergrove. If Elias succeeded, magic would vanish from the world forever. Maya joined forces with Professor Finch and Leo, a brave apprentice wizard, to protect the Heartstone.

Together, they deciphered magical clues hidden around iconic NYC landmarks, from enchanted riddles at the Statue of Liberty to secret spells beneath the Brooklyn Bridge. After a thrilling chase through the underground tunnels of Grand Central Terminal, Maya confronted Elias atop the Empire State Building.

Drawing courage from the legacy of her grandmother, Maya unleashed powerful magic she never knew she possessed, binding Elias and securing the Heartstone. Magic surged brightly once more throughout Eldergrove, and Maya knew her adventures were only beginning.
"""

questions = [
    "Where is the magical neighborhood of ElderGrove located?",
    "How did Maya Torres discover she had magical abilities?",
    "Who is Professor Isadora Finch, and what is her role in Eldergrove?",
    "List three real NYC landmarks that were mentioned in the story as having magical significance."
]

In [5]:
QUESTION_ANSWERING_PROMPT = """
Task: The user has a list of queires that they would like answered based on a piece of source text. Your job is to answer each of their questions and cite the text that supports your answer.

Output Format:
For each question provide:
- answer - a string holding your response
- cited_text - a list of strings of cited text

Input:
User queries: {user_queries}

Source text: {source_text}
"""

In [6]:
message_history = [
    {
        "role": "system",
        "content": "You are an expert in patent law critiquing a patent examiner's analysis from a non-final rejection"
    },
    {
        "role": "user",
        "content": QUESTION_ANSWERING_PROMPT.format(
            user_queries=questions,
            source_text=source_text
        )
    }
]

llm_usage = ModelUsageAsync()

model_response = await call_openai_structured(
    openai_client=openai_client,
    model="o4-mini",
    messages=message_history,
    reasoning_effort="high",
    llm_usage=llm_usage
)

print(f"Tokens used: {await llm_usage.get_tokens_used()}")
print(f"Total cost: ${await llm_usage.get_cost()}")


Tokens used: 4309
Total cost: $0.017230400000000003


In [7]:
question_answers_response = model_response.choices[0].message.content
print(question_answers_response)

1. Where is the magical neighborhood of Eldergrove located?  
- answer: "It is hidden in the bustling heart of New York City, concealed behind an ancient oak in Central Park."  
- cited_text:  
  - "In the bustling heart of New York City, hidden from ordinary eyes, lay the enchanting neighborhood of Eldergrove, concealed behind an ancient oak in Central Park."

2. How did Maya Torres discover she had magical abilities?  
- answer: "She discovered her magical abilities when she confronted Elias Grimwald atop the Empire State Building and, drawing on her grandmother’s legacy, unleashed powerful magic she never knew she possessed."  
- cited_text:  
  - "Drawing courage from the legacy of her grandmother, Maya unleashed powerful magic she never knew she possessed, binding Elias and securing the Heartstone."

3. Who is Professor Isadora Finch, and what is her role in Eldergrove?  
- answer: "She is the wise sorcerer who guards the shimmering gateway to Eldergrove and serves as Maya’s mento

In [8]:
class Answer(BaseModel):
    question: str
    answer: str
    cited_text: list[str]

class AllAnswers(BaseModel):
    answers_list: list[Answer]

FORMATTING_PROMPT = """
Task: An upstream model has answered some user questions. Your job is to format the response into a structured output.

Formatting Instructions:
For each question, provide:
- question - a string holding the question that was asked
- answer - a string holding your response
- cited_text - a list of strings of cited text

Input:
Upstream model response: {upstream_model_response}
"""

In [9]:
message_history = [
    {
        "role": "system",
        "content": "You are a meticulous proofreader"
    },
    {
        "role": "user",
        "content": FORMATTING_PROMPT.format(
            upstream_model_response=question_answers_response
        )
    }
]

llm_usage = ModelUsageAsync()

formatted_response = await call_openai_structured(
    openai_client=openai_client,
    model="gpt-4.1",
    messages=message_history,
    response_format=AllAnswers,
    llm_usage=llm_usage
)

print(f"Tokens used: {await llm_usage.get_tokens_used()}")
print(f"Total cost: ${await llm_usage.get_cost()}")

Tokens used: 927
Total cost: $0.003966


In [10]:
structured_question_answers_response = formatted_response.choices[0].message.parsed

In [11]:
for question_number, answer_object in enumerate(structured_question_answers_response.answers_list):
    print(f"Question {question_number + 1}: {answer_object.question}")
    print(f"Answer: {answer_object.answer}")
    print(f"Cited Text: {answer_object.cited_text}")
    print("\n")

Question 1: Where is the magical neighborhood of Eldergrove located?
Answer: It is hidden in the bustling heart of New York City, concealed behind an ancient oak in Central Park.
Cited Text: ['In the bustling heart of New York City, hidden from ordinary eyes, lay the enchanting neighborhood of Eldergrove, concealed behind an ancient oak in Central Park.']


Question 2: How did Maya Torres discover she had magical abilities?
Answer: She discovered her magical abilities when she confronted Elias Grimwald atop the Empire State Building and, drawing on her grandmother’s legacy, unleashed powerful magic she never knew she possessed.
Cited Text: ['Drawing courage from the legacy of her grandmother, Maya unleashed powerful magic she never knew she possessed, binding Elias and securing the Heartstone.']


Question 3: Who is Professor Isadora Finch, and what is her role in Eldergrove?
Answer: She is the wise sorcerer who guards the shimmering gateway to Eldergrove and serves as Maya’s mentor, g